## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [15]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal([784, 128]))  # 输入层到隐藏层的权重
        self.b1 = tf.Variable(tf.zeros([128]))  # 隐藏层的偏置
        self.W2 = tf.Variable(tf.random.normal([128, 10]))  # 隐藏层到输出层的权重
        self.b2 = tf.Variable(tf.zeros([10]))  # 输出层的偏置
        
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x = tf.reshape(x, [-1, 784])
        hidden = tf.matmul(x, self.W1) + self.b1
        hidden = tf.nn.relu(hidden)  
        
        # 输出层计算
        logits = tf.matmul(hidden, self.W2) + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [16]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [17]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 108.17925 ; accuracy 0.092083335
epoch 1 : loss 96.897736 ; accuracy 0.0952
epoch 2 : loss 90.587074 ; accuracy 0.0994
epoch 3 : loss 85.802376 ; accuracy 0.104383335
epoch 4 : loss 81.78683 ; accuracy 0.1096
epoch 5 : loss 78.24902 ; accuracy 0.114033334
epoch 6 : loss 75.057335 ; accuracy 0.11913333
epoch 7 : loss 72.15228 ; accuracy 0.12325
epoch 8 : loss 69.49371 ; accuracy 0.12728333
epoch 9 : loss 67.07292 ; accuracy 0.13235
epoch 10 : loss 64.88162 ; accuracy 0.1364
epoch 11 : loss 62.905357 ; accuracy 0.14153333
epoch 12 : loss 61.11212 ; accuracy 0.14651667
epoch 13 : loss 59.48572 ; accuracy 0.15141666
epoch 14 : loss 57.99831 ; accuracy 0.1568
epoch 15 : loss 56.62179 ; accuracy 0.1618
epoch 16 : loss 55.3318 ; accuracy 0.16753334
epoch 17 : loss 54.11023 ; accuracy 0.17358333
epoch 18 : loss 52.94513 ; accuracy 0.17906667
epoch 19 : loss 51.82856 ; accuracy 0.18505
epoch 20 : loss 50.75488 ; accuracy 0.19115
epoch 21 : loss 49.720108 ; accuracy 0.1973
epoch 2